In [22]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
import seaborn as sns 
import matplotlib.pyplot as plt 

In [66]:
B1=pd.read_csv("Building1n.csv")
B2=pd.read_csv("Building2n.csv")
B3=pd.read_csv("Building3n.csv")
B4=pd.read_csv("Building4n.csv")
B5=pd.read_csv("Building5n.csv")

In [ ]:
#creating new features
#sum_sub_meters
B1['sum_sub_meters'] = B1.apply(lambda row: row.sub_meter_1 + row.sub_meter_2, axis =1) 

#working hours
def working_h(x):
    if(x>=6.0 and x<=19.0):
        return 1
    else:
        return 0
B1['working hours']=B1["time"].apply(lambda  x:working_h(x))

#lag_main_meter
for i in range (1,15):
    df1=B1.shift(i*96)["main_meter"]
    B1["lag_mm_"+str(i)]=df1
B1=B1.dropna()

#lag_submeter_1
s1=pd.read_csv("Building1n.csv")
for i in range (1,15):
    df1=s1.shift(i*96)["sub_meter_1"]
    s1["lag_s1_"+str(i)]=df1
s1=s1.dropna()
s1=s1.drop(['Unnamed: 0', 'month', 'day', 'main_meter', 'sub_meter_1',
       'sub_meter_2', 'building_number', 'time', 'weekday'],axis=1)

#lag_submeter_2
s2=pd.read_csv("Building1n.csv")
for i in range (1,15):
    df1=s2.shift(i*96)["sub_meter_2"]
    s2["lag_s2_"+str(i)]=df1
s2=s2.dropna()
s2=s2.drop(['Unnamed: 0', 'month', 'day', 'main_meter', 'sub_meter_1',
       'sub_meter_2', 'building_number', 'time', 'weekday',],axis=1)
#concatinating all lags in a single dataframe
#B1=pd.concat

In [71]:
# FAST FOURIER TRANSFORMATION 
build2=pd.read_csv("Building1n.csv")
extra3=[]
mo=[]
import pylab as pl
from numpy import fft
extra2={0}
def fourierExtrapolation(x, n_predict):
    n = x.size
    n_harm = 2                   # number of harmonics in model
    t = np.arange(0, n)
    p = np.polyfit(t, x, 1)         # find linear trend in x
    x_notrend = x - p[0] * t        # detrended x
    x_freqdom = fft.fft(x_notrend)  # detrended x in frequency domain
    f = fft.fftfreq(n)              # frequencies
    indexes = list(range(n))
    # sort indexes by frequency, lower -> higher
    indexes.sort(key = lambda i: np.absolute(f[i]))
 
    t = np.arange(0, n + n_predict)
    restored_sig = np.zeros(t.size)
    for i in indexes[:1 + n_harm * 2]:
        ampli = np.absolute(x_freqdom[i]) / n   # amplitude
        phase = np.angle(x_freqdom[i])          # phase
        restored_sig += ampli * np.cos(2 * np.pi * f[i] * t + phase)
    return restored_sig + p[0] * t
    
def calc(x2,x):
    
    n_predict = 4
    if(x>=4):
        n_predict = n_predict + 1
    extrapolation = fourierExtrapolation(x2, n_predict)
    mo.append(extrapolation)
    p=extrapolation[-4]
    extra3.append(p)
    
for x in range(0,7):
    for y in range (0,24):
        for k in range (0,4):
            data = build2[(build2['weekday']==x) & (build2['time']==y+(k/4)) & (build2['month']!=12)]

            x2=data['sub_meter_2']
            calc(x2,x)



week_fft_s2 = []
for j in range(0,len(mo[-1])):
    for i in range(0,len(mo)):
        try:
            week_fft_s2.append(mo[i][j])
        except:
            continue

In [ ]:
#correlation between features
corrmat=B1.corr()
f, ax = plt.subplots(figsize =(9, 8)) 
sns.heatmap(corrmat, ax = ax, cmap ="YlGnBu", linewidths = 0.1) 

In [10]:
#ANOVA TEST
s1=stats.f_oneway(B1['main_meter'][B1['weekday'] == 0],
               B1['main_meter'][B1['weekday'] == 1],
               B1['main_meter'][B1['weekday'] == 2],
               B1['main_meter'][B1['weekday'] == 3],
               B1['main_meter'][B1['weekday'] == 4],
               B1['main_meter'][B1['weekday'] == 5],
               B1['main_meter'][B1['weekday'] == 6])

s2=stats.f_oneway(B1['sub_meter_1'][B1['weekday'] == 0],
               B1['sub_meter_1'][B1['weekday'] == 1],
               B1['sub_meter_1'][B1['weekday'] == 2],
               B1['sub_meter_1'][B1['weekday'] == 3],
               B1['sub_meter_1'][B1['weekday'] == 4],
               B1['sub_meter_1'][B1['weekday'] == 5],
               B1['sub_meter_1'][B1['weekday'] == 6])

s3=stats.f_oneway(B1['sub_meter_2'][B1['weekday'] == 0],
               B1['sub_meter_2'][B1['weekday'] == 1],
               B1['sub_meter_2'][B1['weekday'] == 2],
               B1['sub_meter_2'][B1['weekday'] == 3],
               B1['sub_meter_2'][B1['weekday'] == 4],
               B1['sub_meter_2'][B1['weekday'] == 5],
               B1['sub_meter_2'][B1['weekday'] == 6])
print("Anova test over main_meter:",s1)
print("Anova test over sub_meter_1:",s2)
print("Anova test over sub_meter_2:",s3)

Anova test over main_meter: F_onewayResult(statistic=207.96139465707395, pvalue=3.2425815218194906e-260)
Anova test over sub_meter_1: F_onewayResult(statistic=70.46724105609704, pvalue=1.8055479108462347e-87)
Anova test over sub_meter_2: F_onewayResult(statistic=295.62408637485885, pvalue=0.0)


In [ ]:
#T-test 
t1=stats.ttest_ind(B1[B1["Holiday"]==0]["main_meter"],B1[B1["Holiday"]==1]["main_meter"],equal_var=False)
t2=stats.ttest_ind(B1[B1["Working Hour"]==0]["main_meter"],B1[B1["Holiday"]==1]["main_meter"],equal_var=False)

In [ ]:
#EXTRA TREE REGRESSOR

from sklearn.ensemble import ExtraTreesRegressor
from sklearn.datasets import make_classification
from sklearn import preprocessing
from sklearn import utils

lab_enc = preprocessing.LabelEncoder()
extra_tree_forest = ExtraTreesRegressor(n_estimators = 8,
                                        max_features = 2)
X = build1bag[['weekday','Holiday','time','working_hour','month','date','lag_sub_meter_1_1','lag_sub_meter_1_2','lag_sub_meter_1_3','lag_sub_meter_1_4','lag_sub_meter_1_5','lag_sub_meter_1_6','lag_sub_meter_1_7','lag_sub_meter_1_8','lag_sub_meter_1_9','lag_sub_meter_1_10','lag_sub_meter_1_11','lag_sub_meter_1_12','lag_sub_meter_1_13','lag_sub_meter_1_14','summer','winter','other','months_fft_s1','week_fft_s1','fft_day_s1']]
y = build1bag['sub_meter_1']


X.head()
# Training the model
extra_tree_forest.fit(X, y)
 
# Computing the importance of each feature
feature_importance = extra_tree_forest.feature_importances_
 
# Normalizing the individual importances
feature_importance_normalized = np.std([tree.feature_importances_ for tree in extra_tree_forest.estimators_], axis = 0)
plt.rcParams['figure.figsize'] = (30,10)

plt.xticks(rotation=90)
plt.rcParams.update({'font.size': 30})
plt.bar(X.columns, feature_importance_normalized)
plt.xlabel('Feature Labels')
plt.ylabel('Feature Importances')
plt.title('Comparison of different Feature Importances')

plt.show()

In [ ]:
#Ridge Regression
b1=pd.read_csv("Build1.csv")
b1=b1.dropna()
x=b1[[ 'month', 'time', 'weekday', 'Holiday','day',
       'sub_meter_sum', 'working_hour', 'fft_main_meter','mmlag1','mmlag2', 'mmlag3', 'mmlag4', 'mmlag5', 'mmlag6', 'mmlag7',
       'mmlag8', 'mmlag9', 'mmlag10', 'mmlag11', 'mmlag12', 'mmlag13',
       'mmlag14', 'summer', 'winter', 'other' ]]
y=b1[['main_meter']]

x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(x, y ,test_size=0.25)
x_train.columns
a=[]
m=y_test
a.append(m.values)
a[0] = [x[0] for x in a[0]]

rr = Ridge(alpha=0.01)
rr.fit(x_train, y_train) 
pred_train_rr= rr.predict(x_train)

pred_test_rr= rr.predict(x_test)
print("r2_score_main_meter",r2_score(y_test, pred_test_rr))
p=[]
m=list(pred_test_rr)
p.append(m)
p[0]= [x[0] for x in p[0]]

x=b1[['month', 'day', 'time', 'weekday', 'Holiday', 
       'sub_meter_sum', 'working_hour', 'fft_main_meter','s2lag1',
       's2lag2', 's2lag3', 's2lag4', 's2lag5', 's2lag6', 's2lag7',
       's2lag8', 's2lag9', 's2lag10', 's2lag11', 's2lag12', 's2lag13','summer', 'winter', 'other']]
y=b1[['sub_meter_2']]
y=np.cbrt(y)

x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(x, y ,test_size=0.25)
x_train.columns
m=y_test
a.append(m.values)
a[1] = [x[0] for x in a[1]]

rr = Ridge(alpha=0.01)
rr.fit(x_train, y_train) 
pred_train_rr= rr.predict(x_train)

pred_test_rr= rr.predict(x_test)
print("r2_sub_sub_2",r2_score(y_test, pred_test_rr))
m=list(pred_test_rr)
p.append(m)
p[1]= [x[0] for x in p[1]]

x=b1[[ 'month', 'day',  'time', 'weekday', 'Holiday',
       'sub_meter_sum', 'fft_main_meter', 'working_hour','s1lag1',
       's1lag2', 's1lag3', 's1lag4', 's1lag5', 's1lag6', 's1lag7',
       's1lag8', 's1lag9', 's1lag10', 's1lag11', 's1lag12', 's1lag13','s1lag14','summer', 'winter', 'other']]
y=b1[['sub_meter_1']]
y=np.cbrt(y)

x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(x, y ,test_size=0.25)
x_train.columns
m=y_test
a.append(m.values)
a[2] = [x[0] for x in a[2]]

rr = Ridge(alpha=0.01)
rr.fit(x_train, y_train) 
pred_train_rr= rr.predict(x_train)

pred_test_rr= rr.predict(x_test)
print("r2_score_sub_meter_1",r2_score(y_test, pred_test_rr))
m=list(pred_test_rr)
p.append(m)
p[2]= [x[0] for x in p[2]]

m2=[]
k=0.0069314718
m2.append(b1["main_meter"].mean())
m2.append(b1["sub_meter_1"].mean())
m2.append(b1["sub_meter_2"].mean())
# Evaluation metric
sum=0
import math
for i in range(3):
    y=0
    x=0
    for j in range(len(p[0])):
        x = (p[i][j]-a[i][j])**2
        y += x*math.exp((-1)*k*b1.iloc[j,3])
    z=y**(1/2)
    sum+= (1/m2[i])*z
e=(1/3)*sum
e

In [ ]:
#lasso Regression
b1=pd.read_csv("b5_final_lag.csv")
b1=b1.dropna()
x=b1[[ 'month', 'time', 'weekday', 'Holiday','date',
       'sub_meter_sum', 'working_hour', 'fft_main_meter','lag_main_meter_1', 'lag_main_meter_2', 'lag_main_meter_3',
       'lag_main_meter_4', 'lag_main_meter_5', 'lag_main_meter_6',
       'lag_main_meter_7', 'lag_main_meter_8', 'lag_main_meter_9',
       'lag_main_meter_10', 'lag_main_meter_11', 'lag_main_meter_12',
       'lag_main_meter_13', 'lag_main_meter_14' ]]
y=b1[['main_meter']]

x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(x, y ,test_size=0.25)
x_train.columns
a=[]
p = []
m=y_test
a.append(m.values)
a[0] = [x[0] for x in a[0]]

model_lasso = Lasso(alpha=0.02)
model_lasso.fit(x_train, y_train) 
pred_train_lasso= model_lasso.predict(x_train)

pred_test_lasso= model_lasso.predict(x_test)
print("r2_score_mm",r2_score(y_test, pred_test_lasso))
m=list(pred_test_lasso)
p.append(list(m))

x=b1[['month', 'date', 'time', 'weekday', 'Holiday', 
       'sub_meter_sum', 'working_hour', 'fft_main_meter','lag_sub_meter_2_1', 'lag_sub_meter_2_2',
       'lag_sub_meter_2_3', 'lag_sub_meter_2_4', 'lag_sub_meter_2_5',
       'lag_sub_meter_2_6', 'lag_sub_meter_2_7', 'lag_sub_meter_2_8',
       'lag_sub_meter_2_9', 'lag_sub_meter_2_10', 'lag_sub_meter_2_11',
       'lag_sub_meter_2_12', 'lag_sub_meter_2_13', 'lag_sub_meter_2_14']]
y=b1[['sub_meter_2']]
y=np.cbrt(y)

x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(x, y ,test_size=0.25)
x_train.columns
m=y_test
a.append(m.values)
a[1] = [x[0] for x in a[1]]

model_lasso = Lasso(alpha=0.02)
model_lasso.fit(x_train, y_train) 
pred_train_lasso= model_lasso.predict(x_train)

pred_test_lasso= model_lasso.predict(x_test)
print("r2_score_s2",r2_score(y_test, pred_test_lasso))

m=list(pred_test_lasso)
p.append(m)

x=b1[[ 'month', 'date',  'time', 'weekday', 'Holiday',
       'sub_meter_sum', 'fft_main_meter', 'working_hour','lag_sub_meter_1_1','lag_sub_meter_1_2', 'lag_sub_meter_1_3', 'lag_sub_meter_1_4',
       'lag_sub_meter_1_5', 'lag_sub_meter_1_6', 'lag_sub_meter_1_7',
       'lag_sub_meter_1_8', 'lag_sub_meter_1_9', 'lag_sub_meter_1_10',
       'lag_sub_meter_1_11', 'lag_sub_meter_1_12', 'lag_sub_meter_1_13','lag_sub_meter_1_14']]
y=b1[['sub_meter_1']]
y=np.cbrt(y)

x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(x, y ,test_size=0.25)
x_train.columns
m=y_test
a.append(m.values)
a[2] = [x[0] for x in a[2]]

model_lasso = Lasso(alpha=0.02)
model_lasso.fit(x_train, y_train) 
pred_train_lasso= model_lasso.predict(x_train)

pred_test_lasso= model_lasso.predict(x_test)
print("r2_score_s1",r2_score(y_test, pred_test_lasso))

m=list(pred_test_lasso)
p.append(m)

m2=[]
k=0.0069314718
m2.append(b1["main_meter"].mean())
m2.append(b1["sub_meter_1"].mean())
m2.append(b1["sub_meter_2"].mean())
# Evaluation metric
sum=0
import math
for i in range(3):
    y=0
    x=0
    for j in range(len(p[0])):
        x = (p[i][j]-a[i][j])**2
        y += x*math.exp((-1)*k*b1.iloc[j,3])
    z=y**(1/2)
    sum+= (1/m2[i])*z
e=(1/3)*sum
e

In [ ]:
#Random Forest
def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[feature_to_encode],prefix=feature_to_encode)
    res = original_dataframe.drop([feature_to_encode],axis = 1)
    res = pd.concat([res, dummies], axis=1)
    return(res)
b_1= encode_and_bind(b1, 'weekday')

b_1['sub_meter_2']=np.cbrt(b_1['sub_meter_2'])

#creating a training set with 80% of given datas:

import random
a=25056
b=0.80*25056
x=random.sample(range(0, int(a)),int(b))
dftrain=b_1.iloc[x]


#creating test set:
dftest = b_1[~b_1.index.isin(dftrain.index)]
#main meter
X_train = dftrain[['month','time', 'year', 'sub_meter_sum','Holiday','working_hour','summer', 'winter', 'other','weekday_0', 'weekday_1', 'weekday_2','weekday_3', 'weekday_4', 'weekday_5', 'weekday_6']]
X_test = dftest[['month','time', 'year', 'sub_meter_sum','Holiday','working_hour','summer', 'winter', 'other','weekday_0', 'weekday_1', 'weekday_2','weekday_3', 'weekday_4', 'weekday_5', 'weekday_6']]

y_train = dftrain['main_meter']
y_test=dftest['main_meter']

'''from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)'''



from sklearn import model_selection
from sklearn.ensemble import RandomForestRegressor

num_instances = len(X_train)

num_trees = 160
max_features = 'auto'
model = RandomForestRegressor(n_estimators=160, random_state=None)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

import sklearn.metrics as metrics


print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('R2_score:{}'.format(model.score(X_test,y_test)))
y_pred_main_1=y_pred
y_test_main_1=y_test


#sub_meter 1
X_train = dftrain[['month','time', 'lag_sub_meter_1_1', 'lag_sub_meter_1_2',
       'lag_sub_meter_1_3', 'lag_sub_meter_1_4', 'lag_sub_meter_1_5',
       'lag_sub_meter_1_6', 'lag_sub_meter_1_7', 'lag_sub_meter_1_8',
       'lag_sub_meter_1_9', 'lag_sub_meter_1_10', 'lag_sub_meter_1_11',
       'lag_sub_meter_1_12', 'lag_sub_meter_1_13', 'lag_sub_meter_1_14','Holiday','working_hour','summer', 'winter', 'other','weekday_0', 'weekday_1', 'weekday_2','weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'Holiday', 'working_hour']]
X_test = dftest[['month','time', 'lag_sub_meter_1_1', 'lag_sub_meter_1_2',
       'lag_sub_meter_1_3', 'lag_sub_meter_1_4', 'lag_sub_meter_1_5',
       'lag_sub_meter_1_6', 'lag_sub_meter_1_7', 'lag_sub_meter_1_8',
       'lag_sub_meter_1_9', 'lag_sub_meter_1_10', 'lag_sub_meter_1_11',
       'lag_sub_meter_1_12', 'lag_sub_meter_1_13', 'lag_sub_meter_1_14','Holiday','working_hour','summer', 'winter', 'other','weekday_0', 'weekday_1', 'weekday_2','weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'Holiday', 'working_hour']]
y_train = dftrain['sub_meter_1']
y_test=dftest['sub_meter_1']

from sklearn import model_selection
from sklearn.ensemble import RandomForestRegressor

num_instances = len(X_train)

num_trees = 160
max_features = 'auto'
model = RandomForestRegressor(n_estimators=160, random_state=None)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

import sklearn.metrics as metrics


print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('R2_score:{}'.format(model.score(X_test,y_test)))
y_pred_s1_1=y_pred
y_test_s1_1=y_test


#sub_meter 2
X_train = dftrain[['month','time',  'lag_sub_meter_2_1', 'lag_sub_meter_2_2', 'lag_sub_meter_2_3',
       'lag_sub_meter_2_4', 'lag_sub_meter_2_5', 'lag_sub_meter_2_6',
       'lag_sub_meter_2_7', 'lag_sub_meter_2_8', 'lag_sub_meter_2_9',
       'lag_sub_meter_2_10', 'lag_sub_meter_2_11', 'lag_sub_meter_2_12',
       'lag_sub_meter_2_13', 'lag_sub_meter_2_14','weekday_0', 'weekday_1', 'weekday_2','weekday_3', 'weekday_4', 'weekday_5', 'weekday_6','Holiday','working_hour','summer', 'winter', 'other','Holiday', 'working_hour']]
X_test= dftest[['month','time',  'lag_sub_meter_2_1', 'lag_sub_meter_2_2', 'lag_sub_meter_2_3',
       'lag_sub_meter_2_4', 'lag_sub_meter_2_5', 'lag_sub_meter_2_6',
       'lag_sub_meter_2_7', 'lag_sub_meter_2_8', 'lag_sub_meter_2_9',
       'lag_sub_meter_2_10', 'lag_sub_meter_2_11', 'lag_sub_meter_2_12',
       'lag_sub_meter_2_13', 'lag_sub_meter_2_14','weekday_0', 'weekday_1', 'weekday_2','weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'Holiday','working_hour','summer', 'winter', 'other','Holiday', 'working_hour']]

y_train = dftrain['sub_meter_2']
y_test=dftest['sub_meter_2']
'''from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)'''




num_instances = len(X_train)

num_trees = 160
max_features = 'auto'
model = RandomForestRegressor(n_estimators=160, random_state=None)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

import sklearn.metrics as metrics


print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('R2_score:{}'.format(model.score(X_test,y_test)))
y_pred_s2_1=y_pred
y_test_s2_1=y_test





y_test_main_1=y_test_main_1.values.reshape((1,5012))
y_test_s1_1=y_test_s1_1.values.reshape((1,5012))
y_test_s2_1=y_test_s2_1.values.reshape((1,5012))
y_test_1=np.concatenate((y_test_main_1, y_test_s1_1,y_test_s2_1), axis=0)

y_pred_main_1=np.reshape(y_pred_main_1,(1,5012))
y_pred_s1_1=np.reshape(y_pred_s1_1,(1,5012))
y_pred_s2_1=np.reshape(y_pred_s2_1,(1,5012))
y_pred_1=np.concatenate((y_pred_main_1, y_pred_s1_1,y_pred_s2_1), axis=0)




m=[]
m.append(dftest['main_meter'].mean())
m.append(dftest['sub_meter_1'].mean())
m.append(dftest['sub_meter_2'].mean())

k=0.0069314718
p=y_pred_1
TEST=y_test_1
# Evaluation metric
arr-dftest['day'].values
arr=arr.reshape((1,5012))
sum=0
import math
for i in range(3):
    y=0
    x=0
    for j in range(len(p[0])):
        x = (p[i][j]-TEST[i][j])**2
        y += x*math.exp((-1)*k*arr[0][j])
    z=y**(1/2)
    sum+= (1/m[i])*z
e=(1/3)*sum

e

In [ ]:
#XGBoost

def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[feature_to_encode],prefix=feature_to_encode)
    res = original_dataframe.drop([feature_to_encode],axis = 1)
    res = pd.concat([res, dummies], axis=1)
    return(res)
b_1= encode_and_bind(b1, 'weekday')



import xgboost as xgb

#b1

#main meter

#creating a training set with 80% of given datas:

import random
a=25056
b=0.80*25056
x=random.sample(range(0, int(a)),int(b))
dftrain=b_1.iloc[x]


#creating test set:
dftest = b_1[~b_1.index.isin(dftrain.index)]

print('main_meter:')

X_train = dftrain[['month','time', 'sub_meter_sum','fft_main_meter', 'lag_main_meter_1', 'lag_main_meter_2','lag_main_meter_3', 'lag_main_meter_4', 'lag_main_meter_5','lag_main_meter_6', 'lag_main_meter_7', 'lag_main_meter_8','lag_main_meter_9', 'lag_main_meter_10', 'lag_main_meter_11','lag_main_meter_12', 'lag_main_meter_13', 'lag_main_meter_14','weekday_0', 'weekday_1', 'weekday_2','weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'Holiday_0','Holiday_1', 'working_hour_0', 'working_hour_1']].values
X_test = dftest[['month','time',  'sub_meter_sum','fft_main_meter', 'lag_main_meter_1', 'lag_main_meter_2','lag_main_meter_3', 'lag_main_meter_4', 'lag_main_meter_5','lag_main_meter_6', 'lag_main_meter_7', 'lag_main_meter_8','lag_main_meter_9', 'lag_main_meter_10', 'lag_main_meter_11','lag_main_meter_12', 'lag_main_meter_13', 'lag_main_meter_14','weekday_0', 'weekday_1', 'weekday_2','weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'Holiday_0','Holiday_1', 'working_hour_0', 'working_hour_1']].values

y_train = dftrain.iloc[:, 5].values
y_test=dftest.iloc[:, 5].values




from sklearn import model_selection

num_instances = len(X_train)

model = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 15, alpha = 10, n_estimators = 160)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

import sklearn.metrics as metrics


print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('R2_score:{}'.format(metrics.r2_score(y_test,y_pred)))
y_pred_main_1=y_pred
y_test_main_1=y_test

#sub_meter 1
print('sub_meter_1:')
X_train = dftrain[['month','time', 'lag_sub_meter_1_1', 'lag_sub_meter_1_2',
       'lag_sub_meter_1_3', 'lag_sub_meter_1_4', 'lag_sub_meter_1_5',
       'lag_sub_meter_1_6', 'lag_sub_meter_1_7', 'lag_sub_meter_1_8',
       'lag_sub_meter_1_9', 'lag_sub_meter_1_10', 'lag_sub_meter_1_11',
       'lag_sub_meter_1_12', 'lag_sub_meter_1_13', 'lag_sub_meter_1_14','weekday_0', 'weekday_1', 'weekday_2','weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'Holiday_0','Holiday_1', 'working_hour_0', 'working_hour_1']].values
X_test = dftest[['month','time',  'lag_sub_meter_1_1', 'lag_sub_meter_1_2',
       'lag_sub_meter_1_3', 'lag_sub_meter_1_4', 'lag_sub_meter_1_5',
       'lag_sub_meter_1_6', 'lag_sub_meter_1_7', 'lag_sub_meter_1_8',
       'lag_sub_meter_1_9', 'lag_sub_meter_1_10', 'lag_sub_meter_1_11',
       'lag_sub_meter_1_12', 'lag_sub_meter_1_13', 'lag_sub_meter_1_14','weekday_0', 'weekday_1', 'weekday_2','weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'Holiday_0','Holiday_1', 'working_hour_0', 'working_hour_1']].values

y_train = dftrain.iloc[:, 6].values
y_test=dftest.iloc[:, 6].values



from sklearn import model_selection


num_instances = len(X_train)

model = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 15, alpha = 10, n_estimators = 160)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

import sklearn.metrics as metrics


print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('R2_score:{}'.format(metrics.r2_score(y_test,y_pred)))
y_pred_s1_1=y_pred
y_test_s1_1=y_test


#sub_meter 2
print('sub_meter_2:')
X_train = dftrain[['month','time',  'lag_sub_meter_2_1', 'lag_sub_meter_2_2', 'lag_sub_meter_2_3',
       'lag_sub_meter_2_4', 'lag_sub_meter_2_5', 'lag_sub_meter_2_6',
       'lag_sub_meter_2_7', 'lag_sub_meter_2_8', 'lag_sub_meter_2_9',
       'lag_sub_meter_2_10', 'lag_sub_meter_2_11', 'lag_sub_meter_2_12',
       'lag_sub_meter_2_13', 'lag_sub_meter_2_14','weekday_0', 'weekday_1', 'weekday_2','weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'Holiday_0','Holiday_1', 'working_hour_0', 'working_hour_1']].values
X_test= dftest[['month','time',  'lag_sub_meter_2_1', 'lag_sub_meter_2_2', 'lag_sub_meter_2_3',
       'lag_sub_meter_2_4', 'lag_sub_meter_2_5', 'lag_sub_meter_2_6',
       'lag_sub_meter_2_7', 'lag_sub_meter_2_8', 'lag_sub_meter_2_9',
       'lag_sub_meter_2_10', 'lag_sub_meter_2_11', 'lag_sub_meter_2_12',
       'lag_sub_meter_2_13', 'lag_sub_meter_2_14','weekday_0', 'weekday_1', 'weekday_2','weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'Holiday_0','Holiday_1', 'working_hour_0', 'working_hour_1']].values

y_train = dftrain.iloc[:, 7].values
y_test=dftest.iloc[:, 7].values



from sklearn import model_selection


model = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 15, alpha = 10, n_estimators = 160)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

import sklearn.metrics as metrics


print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('R2_score:{}'.format(metrics.r2_score(y_test,y_pred)))
y_pred_s2_1=y_pred
y_test_s2_1=y_test

y_pred_main_1=np.reshape(y_pred_main_1,(1,5012))
y_pred_s1_1=np.reshape(y_pred_s1_1,(1,5012))
y_pred_s2_1=np.reshape(y_pred_s2_1,(1,5012))
y_pred_1=np.concatenate((y_pred_main_1, y_pred_s1_1,y_pred_s2_1), axis=0)


y_test_main_1=np.reshape(y_test_main_1,(1,5012))
y_test_s1_1=np.reshape(y_test_s1_1,(1,5012))
y_test_s2_1=np.reshape(y_test_s2_1,(1,5012))
y_test_1=np.concatenate((y_test_main_1, y_test_s1_1,y_test_s2_1), axis=0)
m=[]
m.append(dftest['main_meter'].mean())
m.append(dftest['sub_meter_1'].mean())
m.append(dftest['sub_meter_2'].mean())

k=0.0069314718
p=y_pred_1
TEST=y_test_1
# Evaluation metric
arr-dftest['day'].values
arr=arr.reshape((1,5012))
sum=0
import math
for i in range(3):
    y=0
    x=0
    for j in range(len(p[0])):
        x = (p[i][j]-TEST[i][j])**2
        y += x*math.exp((-1)*k*arr[0][j])
    z=y**(1/2)
    sum+= (1/m[i])*z
e=(1/3)*sum

e

